In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
def excel_comparision(Golden_Excel_file,Extracted_Excel_file):
    
    #covert file path into string format
    Excel_file_1 = str(Golden_Excel_file)
    Excel_file_2 = str(Extracted_Excel_file)
    
    
    excel_1 = pd.ExcelFile(Excel_file_1)
    excel_2 = pd.ExcelFile(Excel_file_2)

    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('excel_comparison.xlsx', engine='xlsxwriter')

    #make the result data frame with shape is equel to df_1.shape
    for sheet in range(len(excel_1.sheet_names)): #for loop on number of sheets present innexcel file
        df_1 = pd.read_excel(excel_1, sheet_name= sheet)
        df_2 = pd.read_excel(excel_2, sheet_name= sheet)

        result_df = pd.DataFrame(data = np.NaN, index = df_1.index,columns=df_1.columns)
        for i in df_1.index:
            for j in df_2.index:
                # if statement checks the key('Name'+ DOB) of df_1 row with every row in df_2 
                if (df_1.iloc[i,:2].apply(lambda x:str(x).strip()) ==df_2.iloc[j,:2].apply(lambda x:str(x).strip())).all():
                    #appending the value TRUE, FALSE by checking series of i'th row of df_1 to series of j'th row of df_2
                    result_df.iloc[i,:] = (df_1.iloc[i,:].apply(lambda x:str(x).strip()) == df_2.iloc[j,:].apply(lambda x:str(x).strip())).values
                    break
                else:
                    result_df.iloc[i,:] = "key not found"
        result_df.to_excel(writer, sheet_name = 'Sheet{}'.format(sheet+1))
    writer.save()


In [ ]:
excel_comparision('excel_1.xlsx', 'excel_2.xlsx')

In [ ]:
def excel_Key_comparison(Golden_Excel_file,Extracted_Excel_file):
    
    #covert file path into string format
    Excel_file_1 = str(Golden_Excel_file)
    Excel_file_2 = str(Extracted_Excel_file)
    
    
    excel_1 = pd.ExcelFile(Excel_file_1)
    excel_2 = pd.ExcelFile(Excel_file_2)


    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('excel_key_comparison.xlsx', engine='xlsxwriter')

    for sheet in range(len(excel_1.sheet_names)): #for loop on number of sheets present innexcel file
        df_1 = pd.read_excel(excel_1, sheet_name= sheet)
        df_2 = pd.read_excel(excel_2, sheet_name= sheet)


        result2_df = pd.DataFrame(data= 0, index= df_1.index, columns=['Number of time key_occured_in_extracted_data'])
        for i in df_1.index:
            for j in df_2.index:
                if (df_1.iloc[i,:2] == df_2.iloc[j,:2]).all(): #this will check key whether present or not
                    result2_df.iloc[i,0] = result2_df.iloc[i,0]+1 #if present it will add to our count
                    
        result2_df.to_excel(writer, sheet_name = 'Sheet{}'.format(sheet+1))
    writer.save()



In [ ]:
excel_Key_comparison('excel_1.xlsx', 'excel_2.xlsx')

In [ ]:
def not_founded_keys(Golden_Excel_file,Extracted_Excel_file):
    
    #covert file path into string format
    Excel_file_1 = str(Golden_Excel_file)
    Excel_file_2 = str(Extracted_Excel_file)
    
    
    excel_1 = pd.ExcelFile(Excel_file_1)
    excel_2 = pd.ExcelFile(Excel_file_2)


    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('Not_founded_key.xlsx', engine='xlsxwriter')

    for sheet in range(len(excel_1.sheet_names)): #for loop on number of sheets present innexcel file
        df_1 = pd.read_excel(excel_1, sheet_name= sheet)
        df_2 = pd.read_excel(excel_2, sheet_name= sheet)
        

        # not_found_keys are the keys from df_2(extracted) which are not found in df_1(golden data)
        not_found_key = [] #create empty list
        for i in df_2.index:
            for j in df_1.index:
                if (df_2.iloc[i,:2] == df_1.iloc[j,:2]).all():
                    break
                else:
                    pass
            else:
                not_found_key.append(df_2.iloc[i,:2]) #append the not founded key to list

        result3_df= pd.DataFrame(data=not_found_key) #convert list data frame , so we can convert it to excel
        
        result3_df.to_excel(writer, sheet_name = 'Sheet{}'.format(sheet+1)) #convert data frame to excel, sheetwise
    
    writer.save()

In [ ]:
not_founded_keys('excel_1.xlsx', 'excel_2.xlsx')

In [53]:
class Excel_comparison:
    count = None
    
    def __init__(self,Golden_Excel_file,Extracted_Excel_file):
        self.Golden_Excel_file    = Golden_Excel_file
        self.Extracted_Excel_file = Extracted_Excel_file
        self.excel_1 = pd.ExcelFile(self.Golden_Excel_file)    # creaed variables so they can run every time we create object
        self.excel_2 = pd.ExcelFile(self.Extracted_Excel_file)
        #Excel_comparison.count+=1 #'---'#
    
    def excel_comparision(self):

        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('excel_comparison_{}.xlsx'.format(Excel_comparison.count), engine='xlsxwriter')

        #make the result data frame with shape is equel to df_1.shape
        for sheet in range(len(self.excel_1.sheet_names)): #for loop on number of sheets present innexcel file
            df_1 = pd.read_excel(self.excel_1, sheet_name= sheet)
            df_2 = pd.read_excel(self.excel_2, sheet_name= sheet)

            result_df = pd.DataFrame(data = np.NaN, index = df_1.index,columns=df_1.columns)
            for i in df_1.index:
                for j in df_2.index:
                    # if statement checks the key('Name'+ DOB) of df_1 row with every row in df_2 
                    if (df_1.iloc[i,:2].apply(lambda x:str(x).strip()) ==df_2.iloc[j,:2].apply(lambda x:str(x).strip())).all():
                        #appending the value TRUE, FALSE by checking series of i'th row of df_1 to series of j'th row of df_2
                        result_df.iloc[i,:] = (df_1.iloc[i,:].apply(lambda x:str(x).strip()) == df_2.iloc[j,:].apply(lambda x:str(x).strip())).values
                        break
                    else:
                        result_df.iloc[i,:] = "key not found"
            result_df.to_excel(writer, sheet_name = 'Sheet{}'.format(sheet+1))
        writer.save()

    def excel_Key_comparison(self):

        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('excel_key_comparison_{}.xlsx'.format(Excel_comparison.count), engine='xlsxwriter')

        for sheet in range(len(self.excel_1.sheet_names)): #for loop on number of sheets present innexcel file
            df_1 = pd.read_excel(self.excel_1, sheet_name= sheet)
            df_2 = pd.read_excel(self.excel_2, sheet_name= sheet)


            result2_df = pd.DataFrame(data= 0, index= df_1.index, columns=['Number of time key_occured_in_extracted_data'])
            for i in df_1.index:
                for j in df_2.index:
                    if (df_1.iloc[i,:2] == df_2.iloc[j,:2]).all(): #this will check key whether present or not
                        result2_df.iloc[i,0] = result2_df.iloc[i,0]+1 #if present it will add to our count

            result2_df.to_excel(writer, sheet_name = 'Sheet{}'.format(sheet+1))
        writer.save()


    
    def not_founded_keys(self):

        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('Not_founded_key_{}.xlsx'.format(Excel_comparison.count), engine='xlsxwriter')

        for sheet in range(len(self.excel_1.sheet_names)): #for loop on number of sheets present innexcel file
            df_1 = pd.read_excel(self.excel_1, sheet_name= sheet)
            df_2 = pd.read_excel(self.excel_2, sheet_name= sheet)


            # not_found_keys are the keys from df_2(extracted) which are not found in df_1(golden data)
            not_found_key = [] #create empty list
            for i in df_2.index:
                for j in df_1.index:
                    if (df_2.iloc[i,:2] == df_1.iloc[j,:2]).all():
                        break
                    else:
                        pass
                else:
                    not_found_key.append(df_2.iloc[i,:2]) #append the not founded key to list

            result3_df= pd.DataFrame(data=not_found_key) #convert list data frame , so we can convert it to excel

            result3_df.to_excel(writer, sheet_name = 'Sheet{}'.format(sheet+1)) #convert data frame to excel, sheetwise

        writer.save()


In [54]:
golden_data_folder = r'C:\Users\Shrirang\Desktop\Data Science Practice\My Projects\Mahesh Jupyter notebook\Extracted Data'
extracted_data_folder = r'C:\Users\Shrirang\Desktop\Data Science Practice\My Projects\Mahesh Jupyter notebook\Golden Data'

In [55]:
from os import listdir
from os.path import isfile, join
golden_excels = [f for f in listdir(golden_data_folder) if isfile(join(golden_data_folder, f))]
extracted_excels = [f for f in listdir(golden_data_folder) if isfile(join(golden_data_folder, f))]

In [56]:
for i in golden_excels:
    for j in extracted_excels:
        if i == j:
            a = Excel_comparison(i,j)
            Excel_comparison.count = i
            a.excel_Key_comparison()
            a.excel_comparision()
            a.not_founded_keys()
            break
#after runing the methods we should assign 0 to class variable 'count'
Excel_comparison.count=None